## Assignment

In this assignment, you'll continue working with the [U.S. Education Dataset](https://www.kaggle.com/noriuk/us-education-datasets-unification-project/home) from Kaggle. The data gives detailed state level information on the several facets of the state of education on annual basis. To learn more about the data and the column descriptions, you can view the Kaggle link above. You should access the data from the Thinkful database. Below are the credentials you can use to connect to the database:

postgres_user = 'dsbc_student'<br>
postgres_pw = '7\*.8G9QH21'<br>
postgres_host = '142.93.121.174'<br>
postgres_port = '5432'<br>
postgres_db = 'useducation'<br>

Don't forget to apply the most suitable missing value filling techniques you applied in the previous checkpoints to the data. You should provide your answers to the following questions after you handled the missing values.

To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

Say, we want to understand the relationship between the expenditures of the governments and the students' overall success in the math and reading.

1. Create a new score variable from the weighted averages of all score variables in the datasets. **Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores!**.
2. What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?
3. Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?
4. What is the correlation between the overall score variable and the 1st principal component? 
5. If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?


In [31]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
import math
from sqlalchemy import create_engine
from scipy.stats import zscore

from matplotlib.mlab import PCA as mlabPCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

education = pd.read_sql_query('select * from useducation',con=engine)

# no need for an open connection, 
# as we're only doing a single query
engine.dispose()



In [32]:
#Creating the function to clean with imputation based on year.

import math

def general_imputation_year(columns):
        
    """ Function: general_imputation based on a year
        Parameters: a list of columns to clean
        Return: string
    """     
    
    years = education['YEAR'].unique()
    
    for column_name in columns:

        new_column = column_name + '_c'
        education[new_column] = education[column_name]
        
        for year in years:
            
            m=education[education['YEAR']==year][[new_column,'YEAR']].mean()
            mean = m.astype('float')
            
            group = (education['YEAR']==year)
    
            if math.isnan(mean[0]):            
                mean_r = education[new_column].mean()

            else:
                mean_r = education[education['YEAR']==year][new_column].mean()
                
            education.loc[group,new_column] = education.loc[group,new_column].fillna(mean_r)

                
    return 'Imputation Done'

In [33]:
#Cleaning columns with mean per year
general_imputation_year(['TOTAL_REVENUE','FEDERAL_REVENUE','STATE_REVENUE','LOCAL_REVENUE','TOTAL_EXPENDITURE','INSTRUCTION_EXPENDITURE','SUPPORT_SERVICES_EXPENDITURE','OTHER_EXPENDITURE','CAPITAL_OUTLAY_EXPENDITURE','GRADES_PK_G','GRADES_KG_G','GRADES_4_G','GRADES_8_G','GRADES_12_G','GRADES_1_8_G','GRADES_9_12_G','GRADES_ALL_G','AVG_MATH_4_SCORE','AVG_MATH_8_SCORE','AVG_READING_4_SCORE','AVG_READING_8_SCORE'])


# Dropping null values after imputation
education.dropna(inplace=True)

In [34]:
education.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G_c,GRADES_8_G_c,GRADES_12_G_c,GRADES_1_8_G_c,GRADES_9_12_G_c,GRADES_ALL_G_c,AVG_MATH_4_SCORE_c,AVG_MATH_8_SCORE_c,AVG_READING_4_SCORE_c,AVG_READING_8_SCORE_c
209,1996_ALABAMA,ALABAMA,1996,735912.0,3365113.0,334858.0,2138717.0,891538.0,3572283.0,1987018.0,...,57098.0,58305.0,42510.0,469370.0,207980.0,686983.0,211.646974,256.594863,210.920767,260.276441
211,1996_ARIZONA,ARIZONA,1996,764681.0,4143304.0,378917.0,1798363.0,1966024.0,4391555.0,1860007.0,...,62793.0,60514.0,42041.0,511925.0,210289.0,726709.0,217.575940,267.874834,206.529788,263.283638
212,1996_ARKANSAS,ARKANSAS,1996,452907.0,2183384.0,192152.0,1296247.0,694985.0,2297381.0,1264275.0,...,33855.0,37064.0,27613.0,283883.0,131997.0,417860.0,215.846436,261.652745,208.837716,271.834590
213,1996_CALIFORNIA,CALIFORNIA,1996,5460484.0,31282981.0,2603882.0,17061474.0,11617625.0,31561692.0,16880730.0,...,435217.0,406456.0,298669.0,3516572.0,1525618.0,5115423.0,209.129083,262.772478,201.912591,269.197451
214,1996_COLORADO,COLORADO,1996,655679.0,3891203.0,194998.0,1664676.0,2031529.0,4075165.0,2052264.0,...,52524.0,52269.0,37179.0,423510.0,185700.0,621730.0,225.805858,275.608214,221.653243,268.168154


### 1. Create a new score variable from the weighted averages of all score variables in the datasets. **Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores!**.


In [35]:
education["overallScore"] = (education["GRADES_4_G_c"]*((education["AVG_MATH_4_SCORE_c"] + education["AVG_READING_4_SCORE_c"])*0.5) + education["GRADES_8_G_c"]
                                 * ((education["AVG_MATH_8_SCORE_c"] + education["AVG_READING_8_SCORE_c"])*0.5))/(education["GRADES_4_G_c"] + education["GRADES_8_G_c"])

In [36]:
education["overallScore"]

209     235.106342
211     238.321404
212     240.773675
213     234.719834
214     247.750273
215     250.735755
217     214.015451
218     235.094096
220     233.380993
224     253.292493
226     235.566500
227     233.293925
228     249.048856
229     239.768842
230     244.523120
235     251.780473
240     238.196210
242     243.670629
251     240.094503
252     244.699649
255     243.779570
256     246.177765
258     250.543086
416     235.863027
418     239.899103
419     239.528677
420     235.179063
422     252.492779
424     210.794614
426     238.618763
           ...    
1219    249.809059
1220    262.963533
1221    249.775478
1222    259.967436
1223    245.806563
1224    252.857928
1225    250.809964
1226    255.648655
1227    248.120131
1228    261.738022
1229    256.870277
1230    245.644471
1231    252.489083
1232    250.978801
1233    254.336029
1234    253.820885
1235    250.481685
1237    254.141869
1238    253.194849
1239    249.595884
1240    251.452502
1241    249.

### 2. What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?

_INSTRUCTION__EXPENDITURE type is more correlated with new variables than the others_

In [37]:
education[["overallScore", "TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE","SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]].corr()

,overallScore,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
overallScore,1.000000,0.086797,0.100311,0.092225,0.015186,0.001657
TOTAL_EXPENDITURE,0.086797,1.000000,0.989849,0.993285,0.941120,0.928628
INSTRUCTION_EXPENDITURE,0.100311,0.989849,1.000000,0.974872,0.898766,0.885351
SUPPORT_SERVICES_EXPENDITURE,0.092225,0.993285,0.974872,1.000000,0.948014,0.915436
OTHER_EXPENDITURE,0.015186,0.941120,0.898766,0.948014,1.000000,0.921652
CAPITAL_OUTLAY_EXPENDITURE,0.001657,0.928628,0.885351,0.915436,0.921652,1.000000


### 3. Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?

_More than 94% of the variance is explained by the first principal component._


In [39]:
X = education[["INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE","OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]]


# Normalizing data to get all variables with mean = 0 and standard deviation = 1

X = StandardScaler().fit_transform(X)

sklearn_pca = PCA(n_components=1)
education["pca_1"] = sklearn_pca.fit_transform(X)

print(
    'The percentage of total variance in the dataset explained by first component from Sklearn PCA: ',
    sklearn_pca.explained_variance_ratio_
)

The percentage of total variance in the dataset explained by first component from Sklearn PCA:  [0.94312987]


### 4. What is the correlation between the overall score variable and the 1st principal component? 

_The correlation between the overall score variable and the 1st principal component is lower than correlation with other expenditure variables like TOTAL_EXPENDITURE, INSTRUCTION_EXPENDITURE and SUPPORT_SERVICES_EXPENDITURE_

In [40]:
education[["overallScore","pca_1","TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE",
              "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]].corr()

,overallScore,pca_1,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
overallScore,1.000000,0.054225,0.086797,0.100311,0.092225,0.015186,0.001657
pca_1,0.054225,1.000000,0.992014,0.967914,0.988354,0.970155,0.957924
TOTAL_EXPENDITURE,0.086797,0.992014,1.000000,0.989849,0.993285,0.941120,0.928628
INSTRUCTION_EXPENDITURE,0.100311,0.967914,0.989849,1.000000,0.974872,0.898766,0.885351
SUPPORT_SERVICES_EXPENDITURE,0.092225,0.988354,0.993285,0.974872,1.000000,0.948014,0.915436
OTHER_EXPENDITURE,0.015186,0.970155,0.941120,0.898766,0.948014,1.000000,0.921652
CAPITAL_OUTLAY_EXPENDITURE,0.001657,0.957924,0.928628,0.885351,0.915436,0.921652,1.000000


### 5. If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?

_I would prefere using INSTRUCTION__EXPENDITURE and other more correlated variables._